In [1]:
import pandas as pd
import numpy as np
import csv
import re

import matplotlib.pyplot as plt

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import cv2
import json

import skimage.measure
import glob

import rasterio
from rasterio.features import shapes

import matplotlib.patches as mpatches
from shapely.geometry import Point, Polygon, shape, mapping
import shapely
import geopandas as gpd

from matplotlib.path import Path
import laspy
import open3d as o3d
import os

from skopt import BayesSearchCV
from sklearn.neighbors import LocalOutlierFactor

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:

cols = ["file", "filepath", "pct_kept_datapoints", "pct_lost_powerline", "params"]
with open('/home/frederik/results_outliers_o3d_newscore.txt', 'r') as f:
    lines = f.read()
lines = lines.split('\n')[:-1]

results = []
for line in lines:
    match = re.search("\{.*\}", line)
    start, end = match.span()
    params = line[start:end]
    data = line[:start-1].split(',')
    results.append(data+[params])
    
df = pd.DataFrame(results, columns=cols)

df['pct_lost_powerline'] = df['pct_lost_powerline'].astype(float)
df['pct_kept_datapoints'] = df['pct_kept_datapoints'].astype(float)

runs = [] 
for i in df['params'].unique():
    tmpDF = df[df['params'] == i]
    runs.append(tmpDF)
    

In [3]:
alpha = 0.9
best_score = 0
best_run = None

for run in runs:
    mean_kept = np.mean(run['pct_kept_datapoints'])
    mean_lost_powerline = np.mean(run['pct_lost_powerline']) 
    score = 1 - (1-alpha)*mean_kept - alpha*mean_lost_powerline
    
    if score > best_score:
        best_score = score
        best_run = run

In [5]:
best_params = best_run.iloc[0].params.replace("'", '"')
best_params = json.loads(best_params)
best_params

{'nb_neighbors': 19,
 'path': '/home/nxw500/data/',
 'std_ratio': 50,
 'voxel_size': 0.7}

In [19]:
# Other loss function
best_run = None
best_params = None
best_kept = 1
for r in runs:
    if np.sum(r['pct_lost_powerline']) == 0:
        mean_kept = np.mean(r['pct_kept_datapoints'])
        if mean_kept < best_kept:
            best_run = r

best_params = best_run.iloc[0].params.replace("'", '"')
best_params = json.loads(best_params)

In [6]:
for path in glob.glob('/home/frederik/data/new_las/*'):
    las = laspy.read(path, laz_backend=laspy.compression.LazBackend.LazrsParallel)
    point_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1, 0))

    # Create o3d Point Cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_data)
    voxel_down_pcd = pcd.voxel_down_sample(voxel_size=best_params['voxel_size'])

    new_pcd, ind = voxel_down_pcd.remove_statistical_outlier(nb_neighbors=best_params['nb_neighbors'],
                                                                std_ratio=best_params['std_ratio'])
    
    print("Points: ", len(las))
    print("Points lost: ", len(las)-len(ind))
    print("Powerline lost: ", np.sum(las.classification == 14) - np.sum(las[ind].classification == 14))
    
    o3d.visualization.draw_geometries([new_pcd])
    

Points:  12909544
Points lost:  309
Powerline lost:  0
Points:  10824872
Points lost:  262
Powerline lost:  0


KeyboardInterrupt: 

In [42]:
list(las.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'synthetic',
 'key_point',
 'withheld',
 'scan_angle_rank',
 'user_data',
 'point_source_id',
 'gps_time',
 'red',
 'green',
 'blue',
 'HeightAboveGround']

In [46]:
np.sum(las.HeightAboveGround > 50)

26